# 1. Deps

In [1]:
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn

In [2]:
from data import *
from utils import *
from gmf import GMF

In [3]:
USE_GPU = True

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('using device:', device)

using device: cuda


# 2. Data

In [4]:
path = './data/'

In [5]:
M = pd.read_csv(f'{path}full_train.csv')

In [6]:
M.head()

,virusUprot,humanUprot,edge
0,P03433,P49736,1.0
1,P03433,P15311,0.0
2,P03433,P11142,0.0
3,P03433,Q86U42,0.0
4,P03433,P33992,1.0


In [7]:
htoi = loadjson(f'{path}htoi.json')
vtoi = loadjson(f'{path}vtoi.json')
itoh = loadjson(f'{path}itoh.json')
itov = loadjson(f'{path}itov.json')

In [8]:
# hfeats = np.loadtxt(f'{path}hfeats.txt')
vfeats = np.loadtxt(f'{path}vfeats.txt')

In [9]:
data_config = {
    'interactions':M,
    'htoi':htoi,
    'vtoi':vtoi,
#     'hfeats':hfeats,
#     'vfeats':vfeats,
    'hfeats':None,
    'vfeats':None,
    'pct_test':.10,
    'device': device
}

In [10]:
gen = ProteinInteractionGenerator(data_config)

using device:  cuda
Found 12192 positives, and 716794 negatives! 0.01672460102114444
--------
9889 in training set, 0.01674744867717341
1036 in val set, 0.15072627231020133
1267 in test set, 0.017380210976830957


In [11]:
loader = gen.create_train_loader(4)

In [12]:
next(iter(loader))

[tensor([319, 343, 340, 319], device='cuda:0'),
 tensor([1389, 2294,  602, 2632], device='cuda:0'),
 tensor([0., 0., 0., 0.], device='cuda:0', dtype=torch.float64)]

# Model - normal mf

In [13]:
n_v, n_h = len(M['virusUprot']), len(M['humanUprot'])

In [14]:
latent_dim = vfeats.shape[1]

In [15]:
config = {
    'num_virus': n_v,
    'num_human': n_h,
    'latent_dim': latent_dim,
    'sparse': False # set false for now because some optimizers dont work with sparse
}

In [16]:
model = GMF(config)

RuntimeError: $ Torch: not enough memory: you tried to allocate 7GB. Buy new RAM! at /opt/conda/conda-bld/pytorch_1544174967633/work/aten/src/TH/THGeneral.cpp:201